In [1]:
import pandas as pd
import random

In [2]:
# load source data
data_df = pd.read_csv("Peru_2019_AudioMoth_Data_Full.csv")

/Users/victoriacyzhang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# delete samples with short durations
temp_index = data_df[data_df['Duration'] < 60].index
data_df.drop(temp_index, inplace=True)
# delete samples with errors 
data_df = data_df[data_df['Error'].isnull()]

In [4]:
# initialize a list of valid AMs
AMs = {}
for am in data_df.Directory.unique(): # iterate through all possible AMs
    curr = data_df.loc[data_df['Directory'] == am]
    hours = set() # set of hours
    tempDict = {} # dictionary of hour -> list of rows
    for _, row in curr.iterrows(): # iterate through current AM
        value = (pd.to_datetime(row['StartDateTime']).hour) # hour corresponded to current row
        hours.add(value)
        # add row to dictionary
        if value in tempDict.keys():
            tempDict[value].append(row)
        else:
            tempDict[value] = [row]
    # only add current AM if it has enough clips
    if len(hours) == 24:
        AMs[am] = tempDict    

In [7]:
# initialize result list
result = [] 
# pick one random row from each hour
for li in AMs.keys():
    for hour in AMs[li]:
        result.append((random.choice(AMs[li][hour])).to_frame().transpose())

In [8]:
# store result as csv
resultdf = pd.concat(result).reset_index(drop=True)
resultdf.to_csv("result.csv")